In [3]:
# In this api call is being made to chat gpt to check if its a follow up question or not

import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from ipywidgets import HTML, Layout
from IPython.display import display
from openai import OpenAI
import openai

class Chatbot:
    def __init__(self, file_paths, api_key):
        self.file_paths = file_paths
        openai.api_key = api_key
        self.knowledge_base = self.load_data()
        self.documents = self.prepare_documents()
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.vectorizer.fit_transform(self.documents)
        self.full_documents = self.knowledge_base
        self.conversation_history = []
        self.setup_interface()
        client = OpenAI()

    def load_data(self):
        all_data = []
        for file_path in self.file_paths:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                all_data.extend(data)
        return all_data

    def concatenate_text(self, json_obj):
        fields = ['CVE_ID', 'Assigner', 'Description']
        concatenated_text = ' '.join(str(json_obj[field]) for field in fields if field in json_obj)
        return concatenated_text

    def prepare_documents(self):
        return [self.concatenate_text(item) for item in self.knowledge_base]

    def answer_question(self, query):
        query_vec = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_vec, self.tfidf_matrix)[0]
        top_indices = np.argsort(similarities)[-10:][::-1]
        top_documents = [self.full_documents[i] for i in top_indices]
        return top_documents

    def should_use_previous_context(self, question):
        client = OpenAI()
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Determine if the following question is a follow-up question."},
                {"role": "user", "content": question}
            ]
        )
        response = completion.choices[0].message.content.strip().lower()

        return 'yes' in response

    def handle_question(self, _=None):
        question = self.text_input.value
        self.conversation_history.append({"role": "user", "content": question})
        client = OpenAI()
        

        if self.should_use_previous_context(question):
            messages = [
                {"role": "system", "content": "You are an assistant that helps with CVE data. Respond with relevant CVE details."}
            ] + self.conversation_history
        else:
            answers = self.answer_question(question)
            json_string = json.dumps(answers, indent=4)
            messages = [
                {"role": "system", "content": "You are an assistant that helps with CVE data. Respond with relevant CVE details."}
            ] + self.conversation_history + [
                {"role": "assistant", "content": json_string}
            ]

        with self.results_output:
            display(widgets.HTML(f'<b><font color="Blue">Question: </font></b><b><font color="Black">{question}</font></b>'))
            for i in messages:
                print(i)
            try:
                completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=messages
                )
                response = completion.choices[0].message.content
                self.conversation_history.append({"role": "assistant", "content": response})

                pretty_response = f"<pre>{response}</pre>"
                styled_response = f"""
                    <style>
                        pre {{
                            background-color: #f0f0f0;
                            border: 1px solid #ccc;
                            border-radius: 5px;
                            padding: 10px;
                            overflow: auto;
                            font-family: Consolas, monospace;
                            font-size: 13px;
                            line-height: 1.4;
                            word-wrap: break-word;
                            width: 1100px;
                        }}
                    </style>
                    <p>{pretty_response}</p>
                """

                html_widget = HTML(
                    value=styled_response,
                    layout=Layout(width='auto')
                )
                display(widgets.HTML(f'<b><font color="green">Answer:</font></b> '))
                display(html_widget)

            except openai.error.RateLimitError:
                print("API rate limit exceeded. Please try again later.")
            except openai.error.OpenAIError as e:
                print(f"An API error occurred: {e}")
            except openai.error.AuthenticationError:
                print("Authentication failed. Check your API key.")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")

    def on_text_input_submit(self, change):
        self.handle_question(change)

    def setup_interface(self):
        self.text_input = widgets.Text(
            placeholder='Enter your question here',
            description='Question:',
            layout=widgets.Layout(width='300px', height='100px')
        )
        self.text_input.style = {'description_width': 'initial'}
        self.text_input.on_submit(self.on_text_input_submit)
        self.results_output = widgets.Output()
        display(self.text_input, self.results_output)

# Initialize the chatbot with multiple files
file_paths = [
    
    'nvdcve-1.1-2024_updated.json',
 'nvdcve-1.1-2023_updated.json',
 'nvdcve-1.1-2022_updated.json',
 'nvdcve-1.1-2021_updated.json',
 'nvdcve-1.1-2020_updated.json',
 'nvdcve-1.1-2019_updated.json',
 'nvdcve-1.1-2018_updated.json',
 'nvdcve-1.1-2017_updated.json',
 'nvdcve-1.1-2016_updated.json',
 'nvdcve-1.1-2015_updated.json',
 'nvdcve-1.1-2014_updated.json',
 'nvdcve-1.1-2013_updated.json',
 'nvdcve-1.1-2012_updated.json',
 'nvdcve-1.1-2011_updated.json',
 'nvdcve-1.1-2010_updated.json',
 'nvdcve-1.1-2009_updated.json',
 'nvdcve-1.1-2008_updated.json',
 'nvdcve-1.1-2007_updated.json',
 'nvdcve-1.1-2006_updated.json',
 'nvdcve-1.1-2005_updated.json',
 'nvdcve-1.1-2004_updated.json',
 'nvdcve-1.1-2003_updated.json',
 'nvdcve-1.1-2002_updated.json'
 
 ]
api_key = 'your-api-key'
chatbot = Chatbot(file_paths, api_key)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15312\2118515619.py:138: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.text_input.on_submit(self.on_text_input_submit)


Text(value='', description='Question:', layout=Layout(height='100px', width='300px'), placeholder='Enter your …

Output()

In [4]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from ipywidgets import HTML, Layout
from IPython.display import display
from openai import OpenAI
import openai

class Chatbot:
    def __init__(self, file_paths, api_key):
        self.file_paths = file_paths
        openai.api_key = api_key
        self.knowledge_base = self.load_data()
        self.documents = self.prepare_documents()
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.vectorizer.fit_transform(self.documents)
        self.full_documents = self.knowledge_base
        self.cve_index = self.create_cve_index()
        self.conversation_history = []
        self.setup_interface()

    def load_data(self):
        all_data = []
        for file_path in self.file_paths:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                all_data.extend(data)
        return all_data

    def concatenate_text(self, json_obj):
        fields = ['CVE_ID', 'Assigner', 'Description']
        concatenated_text = ' '.join(str(json_obj[field]) for field in fields if field in json_obj)
        return concatenated_text

    def prepare_documents(self):
        return [self.concatenate_text(item) for item in self.knowledge_base]

    def create_cve_index(self):
        cve_index = {}
        for idx, item in enumerate(self.knowledge_base):
            cve_id = item.get('CVE_ID')
            if cve_id:
                cve_index[cve_id] = idx
        return cve_index

    # def answer_question(self, query):
    #     if query in self.cve_index:
    #         index = self.cve_index[query]
    #         return [self.full_documents[index]]
        
    #     query_vec = self.vectorizer.transform([query])
    #     similarities = cosine_similarity(query_vec, self.tfidf_matrix)[0]
    #     top_indices = np.argsort(similarities)[-5:][::-1]
    #     top_documents = [self.full_documents[i] for i in top_indices]
    #     return top_documents

    def answer_question(self, query):
        query_words = query.split()
        found_documents = []

        # Check if any word in the query is a CVE_ID
        for word in query_words:
            if word in self.cve_index:
                index = self.cve_index[word]
                found_documents.append(self.full_documents[index])

        # If any CVE_ID is found, return the corresponding documents
        if found_documents:
            return found_documents

        # If no CVE_ID is found, proceed with TF-IDF similarity search
        similarity_scores = np.zeros(len(self.documents))

        for word in query_words:
            query_vec = self.vectorizer.transform([word])
            similarities = cosine_similarity(query_vec, self.tfidf_matrix)[0]
            similarity_scores += similarities

        top_indices = np.argsort(similarity_scores)[-10:][::-1]
        top_documents = [self.full_documents[i] for i in top_indices]
        return top_documents


    def should_use_previous_context(self, question):
        client = OpenAI()
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Determine if the following question is a follow-up question."},
                {"role": "user", "content": question}
            ]
        )
        response = completion.choices[0].message.content.strip().lower()
        return 'yes' in response

    def handle_question(self, _=None):

        question = self.text_input.value
        self.conversation_history.append({"role": "user", "content": question})
        client = OpenAI()
        if self.should_use_previous_context(question):
            messages = [
                {"role": "system", "content": "You are an assistant that helps with CVE data. Respond with relevant CVE details. Use previous context as well"}
            ] + self.conversation_history
        else:
            answers = self.answer_question(question)
            json_string = json.dumps(answers, indent=4)
            messages = [
                {"role": "system", "content": "You are an assistant that helps with CVE data. Respond with relevant CVE details."}
            ] + self.conversation_history + [
                {"role": "assistant", "content": json_string}
            ]

        with self.results_output:
            display(widgets.HTML(f'<b><font color="Blue">Question: </font></b><b><font color="Black">{question}</font></b>'))
            for i in messages:
                print(i)
            try:
                completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=messages
                )
                response = completion.choices[0].message.content
                self.conversation_history.append({"role": "assistant", "content": response})

                pretty_response = f"<pre>{response}</pre>"
                styled_response = f"""
                    <style>
                        pre {{
                            background-color: #f0f0f0;
                            border: 1px solid #ccc;
                            border-radius: 5px;
                            padding: 10px;
                            overflow: auto;
                            font-family: Consolas, monospace;
                            font-size: 13px;
                            line-height: 1.4;
                            word-wrap: break-word;
                            width: 1100px;
                        }}
                    </style>
                    <p>{pretty_response}</p>
                """

                html_widget = HTML(
                    value=styled_response,
                    layout=Layout(width='auto')
                )
                display(widgets.HTML(f'<b><font color="green">Answer:</font></b> '))
                display(html_widget)

            except openai.error.RateLimitError:
                print("API rate limit exceeded. Please try again later.")
            except openai.error.OpenAIError as e:
                print(f"An API error occurred: {e}")
            except openai.error.AuthenticationError:
                print("Authentication failed. Check your API key.")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")

    def on_text_input_submit(self, change):
        self.handle_question(change)

    def setup_interface(self):
        self.text_input = widgets.Text(
            placeholder='Enter your question here',
            description='Question:',
            layout=widgets.Layout(width='300px', height='100px')
        )
        self.text_input.style = {'description_width': 'initial'}
        self.text_input.on_submit(self.on_text_input_submit)
        self.results_output = widgets.Output()
        display(self.text_input, self.results_output)

# Initialize the chatbot with multiple files
file_paths = [
    
    'nvdcve-1.1-2024_updated.json',
 'nvdcve-1.1-2023_updated.json',
 'nvdcve-1.1-2022_updated.json',
 'nvdcve-1.1-2021_updated.json',
 'nvdcve-1.1-2020_updated.json',
 'nvdcve-1.1-2019_updated.json',
 'nvdcve-1.1-2018_updated.json',
 'nvdcve-1.1-2017_updated.json',
 'nvdcve-1.1-2016_updated.json',
 'nvdcve-1.1-2015_updated.json',
 'nvdcve-1.1-2014_updated.json',
 'nvdcve-1.1-2013_updated.json',
 'nvdcve-1.1-2012_updated.json',
 'nvdcve-1.1-2011_updated.json',
 'nvdcve-1.1-2010_updated.json',
 'nvdcve-1.1-2009_updated.json',
 'nvdcve-1.1-2008_updated.json',
 'nvdcve-1.1-2007_updated.json',
 'nvdcve-1.1-2006_updated.json',
 'nvdcve-1.1-2005_updated.json',
 'nvdcve-1.1-2004_updated.json',
 'nvdcve-1.1-2003_updated.json',
 'nvdcve-1.1-2002_updated.json'
 
 ]
# file_paths = [
    
#     'nvdcve-1.1-2024_updated.json',
#  'nvdcve-1.1-2023_updated.json',
#  ]
api_key = 'your-api-key'
chatbot = Chatbot(file_paths, api_key)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11156\697481720.py:173: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.text_input.on_submit(self.on_text_input_submit)


Text(value='', description='Question:', layout=Layout(height='100px', width='300px'), placeholder='Enter your …

Output()